In [1]:
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

In [2]:
dane = pd.read_csv('train.no-text.csv', low_memory=False)
dane = dane.drop(columns=['title', 'description', 'city', 'lon_city', 'lat_city', 'lon_district', 'lat_district'])
dane.isnull().sum()

created_at_first              0
category                      0
is_business                   0
price                         0
city_district                 0
                           ... 
vicinity_types_sea          996
heating_types_geothermal    998
heating_types_urban         992
heating_types_stove         997
heating_types_biomass       998
Length: 89, dtype: int64

In [3]:
dane.head()

,created_at_first,category,is_business,price,city_district,price[currency],m,rooms_num,market,building_type,...,heating_types_electric,heating_types_heat_pump,access_types_dirt,fence_types_hedge,heating_types_solar_collector,vicinity_types_sea,heating_types_geothermal,heating_types_urban,heating_types_stove,heating_types_biomass
0,2018-10-06 12:20:48,Mieszkanie na sprzedaż,1,389556,OlsztynDajtki,PLN,72.14,4,secondary,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2019-02-18 12:17:51,Mieszkanie na sprzedaż,1,500000,Plewiska,PLN,95.00,4,secondary,ribbon,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2019-03-21 17:31:09,Mieszkanie na sprzedaż,1,238000,BiałystokZielone Wzgórza,PLN,58.90,3,secondary,block,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2018-11-28 23:50:45,Mieszkanie na sprzedaż,1,209000,PoznańŁazarz,PLN,45.00,2,secondary,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2019-01-22 21:22:42,Dom na sprzedaż,0,729000,Wymysły,PLN,195.00,6,secondary,detached,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
#dane_troche_oczyszczone = dane.drop(columns = ['title'])

dane_troche_oczyszczone = dane.iloc[:,np.array(dane.isnull().sum() < len(dane)*0.2)]

kiedy = dane_troche_oczyszczone['created_at_first']
kiedy = [x.split()[0] for x in kiedy]
kiedy = [x.replace("-", "") for x in kiedy]

dane_troche_oczyszczone["created_at_first"] = kiedy
dane_troche_oczyszczone['created_at_first'] = dane_troche_oczyszczone["created_at_first"].astype(int)



C:\Users\zbugo\AppData\Local\Temp\ipykernel_17436\846418060.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dane_troche_oczyszczone["created_at_first"] = kiedy
C:\Users\zbugo\AppData\Local\Temp\ipykernel_17436\846418060.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dane_troche_oczyszczone['created_at_first'] = dane_troche_oczyszczone["created_at_first"].astype(int)


In [9]:
dane_troche_oczyszczone.dtypes

created_at_first      int32
category             object
is_business           int64
price                 int64
city_district        object
price[currency]      object
m                   float64
rooms_num            object
market               object
building_type        object
rent[currency]       object
dtype: object

In [16]:
#def uzupelnianie(df):
    





posortowane_dane = dane_troche_oczyszczone[pd.DataFrame(dane_troche_oczyszczone.isnull().sum().sort_values()).index]



### ZAMIEŃ MORE NA LICZBĘ NP 11!!! wtedy bedzie mozna bedzie sobie uzepelnic lasem losowym ###
### WSZYSTKIE ZMIENNA ZAMIEŃ NA LICZBOWE ŻEBY UZUPEŁNIĆ BRAKI DANYCH ###

gdzie_braki =  np.array(posortowane_dane.isnull().sum())
braki_index = np.where(gdzie_braki != 0)


X = posortowane_dane.iloc[:,0:braki_index[0][0]]
y = posortowane_dane.iloc[:,braki_index[0][0]]

X = pd.get_dummies(X, drop_first=True)

y_proba_tylko_pelne = y[y.isnull() == False]
X_proba_tylko_pelne = X[y.isnull() == False]
X_wybrakowane = X[y.isnull()]

#X_proba_tylko_pelne.get_dummies()
#X_proba_tylko_pelne = pd.get_dummies(X_proba_tylko_pelne, drop_first=True)
#X_proba_tylko_pelne
#X_wybrakowane = pd.get_dummies(X_wybrakowane, drop_first=True)
#X_wybrakowane



#y_proba_tylko_pelne[y_proba_tylko_pelne == 'more'] = 11
#y_proba_tylko_pelne = y_proba_tylko_pelne.astype('int16')

X_train, X_test, y_train, y_test = train_test_split(X_proba_tylko_pelne, y_proba_tylko_pelne, train_size=0.75)

las_uzupelniajacy = RandomForestClassifier(n_estimators=9)

las_uzupelniajacy.fit(X_train, y_train)



#param_grid = {'n_estimators' : np.arange(1, 10)
#}

#gs = GridSearchCV(las_uzupelniajacy, param_grid=param_grid)
#gs.fit(X=X_train, y=y_train)

#gs.best_params_




#X_train.dtypes


#X[y.isnull()]


IndentationError: expected an indented block after function definition on line 1 (3898031963.py, line 8)

In [15]:
las_uzupelniajacy.predict(X_wybrakowane)

array(['6'], dtype=object)

In [ ]:
y_proba_tylko_pelne

In [42]:
print(pd.DataFrame(dane_troche_oczyszczone.isnull().sum().sort_values()))

                    0
created_at_first    0
category            0
is_business         0
price               0
city_district       0
price[currency]     0
m                   0
market              0
rooms_num           1
rent[currency]    142
building_type     168


In [8]:
pd.get_dummies(dane_troche_oczyszczone, drop_first=True, dummy_na=True)

,created_at_first,is_business,price,m,category_Mieszkanie na sprzedaż,category_nan,city_district_Babiczki,city_district_Banino,city_district_Barczewko,city_district_Barczewo,...,building_type_detached,building_type_farm,building_type_house,building_type_infill,building_type_residence,building_type_ribbon,building_type_semi_detached,building_type_tenement,building_type_nan,rent[currency]_nan
0,20181006,1,389556,72.14,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
1,20190218,1,500000,95.00,True,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
2,20190321,1,238000,58.90,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,20181128,1,209000,45.00,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
4,20190122,0,729000,195.00,False,False,False,False,False,False,...,True,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,20180725,1,270000,40.24,True,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,False
996,20181202,1,355000,68.21,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
997,20181115,1,549000,161.50,False,False,False,False,False,False,...,True,False,False,False,False,False,False,False,False,False
998,20180328,1,150000,91.00,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,False


In [110]:
from sklearn.impute import KNNImputer
import pandas as pd

# Przykładowa ramka danych z brakującymi danymi
data = {
    'A': [1, 2, None, 4, 5],
    'B': [6, None, 8, 9, 10]
}
df = pd.DataFrame(data)

# Inicjalizacja i dopasowanie imputera
imputer = KNNImputer(n_neighbors=2)  # określ liczbę sąsiadów (k)
df_imputed = pd.DataFrame(imputer.fit_transform(df), columns=df.columns)

print("Ramka danych przed uzupełnieniem braków:")
print(df)
print("\nRamka danych po uzupełnieniu braków:")
print(df_imputed)

Ramka danych przed uzupełnieniem braków:
     A     B
0  1.0   6.0
1  2.0   NaN
2  NaN   8.0
3  4.0   9.0
4  5.0  10.0

Ramka danych po uzupełnieniu braków:
     A     B
0  1.0   6.0
1  2.0   7.5
2  2.5   8.0
3  4.0   9.0
4  5.0  10.0


In [112]:
RandomForestClassifier?

Init signature:
RandomForestClassifier(
    n_estimators=100,
    *,
    criterion='gini',
    max_depth=None,
    min_samples_split=2,
    min_samples_leaf=1,
    min_weight_fraction_leaf=0.0,
    max_features='sqrt',
    max_leaf_nodes=None,
    min_impurity_decrease=0.0,
    bootstrap=True,
    oob_score=False,
    n_jobs=None,
    random_state=None,
    verbose=0,
    warm_start=False,
    class_weight=None,
    ccp_alpha=0.0,
    max_samples=None,
)
Docstring:     
A random forest classifier.

A random forest is a meta estimator that fits a number of decision tree
classifiers on various sub-samples of the dataset and uses averaging to
improve the predictive accuracy and control over-fitting.
The sub-sample size is controlled with the `max_samples` parameter if
`bootstrap=True` (default), otherwise the whole dataset is used to build
each tree.

For a comparison between tree-based ensemble models see the example
:ref:`sphx_glr_auto_examples_ensemble_plot_forest_hist_grad_boosting_co